In [20]:
#All these packages need to be installed from pip
#These are all for the cluster detection
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics

import scipy #For hierarchical clustering and some visuals
#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics
import seaborn as sns #Makes the graphics look nicer

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

import itertools
import json

import numpy as np
import pandas as pd  
import gensim#For topic modeling
import lucem_illud

now_df = pd.read_table('data/sample.txt' ,header=0, encoding='gb2312', sep=',') 
now_df

,id,text,month,year
0,1705816,"For those of us who have not known war , the ...",11,10
1,4512620,"Kazi Khaleed Ashraf ONCE upon a time , Dhak...",3,10
2,1448476,There is a myth in wide circulation that the ...,4,10
3,1490802,Please turn on JavaScript . Media requires Ja...,5,10
4,1647381,Image caption The death of Mono Jojoy could c...,9,10
...,...,...,...,...
45425,31034526,Can an academic study also double up as a riv...,6,19
45426,50673735,"Glasgow Southside , where Nicola Sturgeon is ...",5,19
45427,31262587,"Rabbi Joachim Prinz , second from the left , ...",9,19
45428,50662844,The global rate of species extinction is ten ...,5,19


In [21]:
now_df = now_df.dropna(axis = 0)

In [22]:
#initialize
nowTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, max_features=1000, 
                                                                  min_df=3, stop_words='english', norm='l2')
#train
nowTFVects = nowTFVectorizer.fit_transform(now_df['text'])
print(nowTFVects.shape)

(45429, 1000)


In [23]:
now_df['tokenized_text'] = now_df['text'].apply(lambda x: lucem_illud.word_tokenize(x))
now_df['normalized_tokens'] = now_df['tokenized_text'].apply(lambda x: lucem_illud.normalizeTokens(x))

<ipython-input-23-43b3eec8bbab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_df['tokenized_text'] = now_df['text'].apply(lambda x: lucem_illud.word_tokenize(x))
<ipython-input-23-43b3eec8bbab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_df['normalized_tokens'] = now_df['tokenized_text'].apply(lambda x: lucem_illud.normalizeTokens(x))


In [24]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

now_df['reduced_tokens'] = now_df['normalized_tokens'].apply(lambda x: dropMissing(x, nowTFVectorizer.vocabulary_.keys()))

<ipython-input-24-6163dab1691c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_df['reduced_tokens'] = now_df['normalized_tokens'].apply(lambda x: dropMissing(x, nowTFVectorizer.vocabulary_.keys()))


In [116]:
now_df

,id,text,month,year,tokenized_text,normalized_tokens,reduced_tokens
0,1705816,"For those of us who have not known war , the ...",11,10,"[For, those, of, us, who, have, not, known, wa...","[know, war, word, soldier, help, understand, r...","[know, war, word, help, understand, day, war, ..."
1,4512620,"Kazi Khaleed Ashraf ONCE upon a time , Dhak...",3,10,"[Kazi, Khaleed, Ashraf, ONCE, upon, a, time, D...","[kazi, khaleed, ashraf, time, dhaka, fine, cit...","[city, street, heart, city, air, sense, commun..."
2,1448476,There is a myth in wide circulation that the ...,4,10,"[There, is, a, myth, in, wide, circulation, th...","[myth, wide, circulation, superiority, free, t...","[free, trade, simply, question, agree, course,..."
3,1490802,Please turn on JavaScript . Media requires Ja...,5,10,"[Please, turn, on, JavaScript, Media, requires...","[turn, javascript, medium, require, javascript...","[turn, play, foreign, secretary, say, place, b..."
4,1647381,Image caption The death of Mono Jojoy could c...,9,10,"[Image, caption, The, death, of, Mono, Jojoy, ...","[image, caption, death, mono, jojoy, cause, fr...","[death, cause, group, military, successful, ga..."
...,...,...,...,...,...,...,...
45425,31034526,Can an academic study also double up as a riv...,6,19,"[Can, an, academic, study, also, double, up, a...","[academic, study, double, rivet, thriller, lon...","[academic, study, london, sense, movement, cen..."
45426,50673735,"Glasgow Southside , where Nicola Sturgeon is ...",5,19,"[Glasgow, Southside, where, Nicola, Sturgeon, ...","[glasgow, southside, nicola, sturgeon, msp, hi...","[high, level, child, poverty, research, daily,..."
45427,31262587,"Rabbi Joachim Prinz , second from the left , ...",9,19,"[Rabbi, Joachim, Prinz, second, from, the, lef...","[rabbi, joachim, prinz, leave, join, leave, re...","[leave, leave, president, photo, march, washin..."
45428,50662844,The global rate of species extinction is ten ...,5,19,"[The, global, rate, of, species, extinction, i...","[global, rate, species, extinction, time, high...","[global, rate, high, average, past, continue, ..."


In [26]:
dictionary = gensim.corpora.Dictionary(now_df['reduced_tokens'])

In [27]:
corpus = [dictionary.doc2bow(text) for text in now_df['reduced_tokens']]

In [28]:
gensim.corpora.MmCorpus.serialize('data/now.mm', corpus)
nowmm = gensim.corpora.MmCorpus('data/now.mm')

In [29]:
nowlda = gensim.models.ldamodel.LdaModel(corpus=nowmm, id2word=dictionary, num_topics=10, alpha='auto', eta='auto')

In [30]:
now1Bow = dictionary.doc2bow(now_df['reduced_tokens'][0])
now1lda = nowlda[now1Bow]

In [31]:
print("The topics of the text: {}".format(now_df['id'][0]))
print("are: {}".format(now1lda))

The topics of the text: 1705816
are: [(0, 0.17430049), (2, 0.09471043), (3, 0.0668286), (4, 0.117352165), (6, 0.019002883), (8, 0.52604234)]


In [41]:
ldaDF = pandas.DataFrame({
        'id' : now_df['id'],
        'year' : now_df['year'],
        'topics' : [nowlda[dictionary.doc2bow(l)] for l in now_df['reduced_tokens']]
    })

In [42]:
ldaDF

,id,year,topics
0,1705816,10,"[(0, 0.17717889), (2, 0.093628466), (3, 0.0695..."
1,4512620,10,"[(0, 0.08159034), (1, 0.3966306), (4, 0.236846..."
2,1448476,10,"[(1, 0.04726045), (2, 0.55008733), (4, 0.39872..."
3,1490802,10,"[(2, 0.2329995), (3, 0.09909913), (5, 0.250480..."
4,1647381,10,"[(0, 0.6065858), (1, 0.09624033), (5, 0.292426..."
...,...,...,...
45425,31034526,19,"[(0, 0.3627134), (1, 0.079046376), (3, 0.11854..."
45426,50673735,19,"[(5, 0.13465354), (8, 0.5176997), (9, 0.344715..."
45427,31262587,19,"[(0, 0.32797137), (2, 0.17207082), (4, 0.37280..."
45428,50662844,19,"[(0, 0.16285183), (1, 0.3746136), (2, 0.146622..."


In [43]:
#Dict to temporally hold the probabilities
topicsProbDict = {i : [0] * len(ldaDF) for i in range(nowlda.num_topics)}

#Load them into the dict
for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

#Update the DataFrame
for topicNum in range(nowlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

ldaDF.head(10)

,id,year,topics,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,1705816,10,"[(0, 0.17717889), (2, 0.093628466), (3, 0.0695...",0.177179,0.000000,0.093628,0.069540,0.135945,0.000000,0.000000,0.000000,0.521584,0.000000
1,4512620,10,"[(0, 0.08159034), (1, 0.3966306), (4, 0.236846...",0.081590,0.396631,0.000000,0.000000,0.236846,0.281470,0.000000,0.000000,0.000000,0.000000
2,1448476,10,"[(1, 0.04726045), (2, 0.55008733), (4, 0.39872...",0.000000,0.047260,0.550087,0.000000,0.398727,0.000000,0.000000,0.000000,0.000000,0.000000
3,1490802,10,"[(2, 0.2329995), (3, 0.09909913), (5, 0.250480...",0.000000,0.000000,0.233000,0.099099,0.000000,0.250480,0.000000,0.000000,0.000000,0.403130
4,1647381,10,"[(0, 0.6065858), (1, 0.09624033), (5, 0.292426...",0.606586,0.096240,0.000000,0.000000,0.000000,0.292427,0.000000,0.000000,0.000000,0.000000
5,1513952,10,"[(0, 0.10036107), (4, 0.6350719), (5, 0.032975...",0.100361,0.000000,0.000000,0.000000,0.635072,0.032975,0.166377,0.000000,0.000000,0.058912
6,1698792,10,"[(0, 0.8850704), (2, 0.074065134), (4, 0.02928...",0.885070,0.000000,0.074065,0.000000,0.029281,0.000000,0.011201,0.000000,0.000000,0.000000
7,1346268,10,"[(2, 0.26740414), (3, 0.22990528), (4, 0.42334...",0.000000,0.000000,0.267404,0.229905,0.423349,0.070669,0.000000,0.000000,0.000000,0.000000
8,1745872,10,"[(0, 0.02059669), (3, 0.67279494), (5, 0.03813...",0.020597,0.000000,0.000000,0.672795,0.000000,0.038139,0.199426,0.019266,0.000000,0.049183
9,4513686,10,"[(1, 0.23998596), (2, 0.031135391), (5, 0.7191...",0.000000,0.239986,0.031135,0.000000,0.000000,0.719193,0.000000,0.000000,0.000000,0.000000


In [55]:
ldaDF_10 = ldaDF[ldaDF["year"] == 10][['topic_%d' %x for x in range(10)]]
ldaDF_10
ldaDFVisN = ldaDF[ldaDF["year"] == 10][['id']]
ldaDFVis = ldaDFV.values
ldaDFVisNames = ldaDFVisN.values

In [65]:
topicsDict = {}
for topicNum in range(nowlda.num_topics):
    topicWords = [w for w, p in nowlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9
0,state,development,party,right,think,say,woman,south,tax,school
1,political,business,election,india,trump,president,man,africa,income,child
2,power,service,policy,law,know,mr,gender,african,rate,education
3,right,need,trade,human,thing,city,work,black,say,student
4,social,sector,economic,social,work,state,say,land,cent,health
5,democracy,work,china,pakistan,want,day,female,anc,high,university
6,society,economic,political,society,way,minister,film,white,pay,say
7,nigeria,growth,vote,indian,come,house,equality,africans,low,work
8,war,public,climate,court,look,tell,day,say,increase,high
9,movement,provide,change,case,good,member,violence,president,wealth,community


# Dynamic topic modeling

In [124]:
corpus= []
docs_per_time_slice = [0] * 10

for i in range(len(now_df)):   
    try:
        corpus.append(now_df['reduced_tokens'][i])
        docs_per_time_slice[int(now_df['year'][i])-10] += 1    
    except:
        continue

        
# just a check if the counts are correctly added
if np.sum(docs_per_time_slice) != len(corpus):
    print(np.sum(docs_per_time_slice), len(corpus))

In [125]:
dictionary = gensim.corpora.Dictionary(corpus)

In [126]:
now_corpus = [dictionary.doc2bow(text) for text in corpus]

In [127]:
gensim.corpora.MmCorpus.serialize('data/dynamic_tm.mm', now_corpus)
nowcorpus = gensim.corpora.MmCorpus('data/dynamic_tm.mm')

In [128]:
from gensim.models import ldaseqmodel

In [129]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=now_corpus, id2word=dictionary, time_slice=docs_per_time_slice, num_topics=10)

C:\Users\xiche\anaconda3\lib\site-packages\gensim\models\ldaseqmodel.py:293: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [130]:
ldaseq.print_topics(time=0)[0]

[('right', 0.025816875671848236),
 ('state', 0.025680110253405965),
 ('social', 0.016496656392233296),
 ('political', 0.015949711713277186),
 ('law', 0.015343364336594162),
 ('society', 0.015025503326079388),
 ('human', 0.012193672927879025),
 ('movement', 0.011507190893687506),
 ('power', 0.009538165375113952),
 ('class', 0.00907589456963248),
 ('war', 0.008889495577802735),
 ('force', 0.00883145695080287),
 ('india', 0.008580950377605147),
 ('justice', 0.008169757047068764),
 ('group', 0.007659872609493186),
 ('case', 0.007228970103421533),
 ('democracy', 0.007074464795762448),
 ('freedom', 0.006396222795191566),
 ('court', 0.006388233766477565),
 ('rule', 0.006385345916866463)]

In [131]:
ldaseq.print_topics(time=1)[0]

[('state', 0.026010662155902586),
 ('right', 0.023839074157816748),
 ('social', 0.018174272769809915),
 ('political', 0.01690378168680194),
 ('society', 0.015238712415773811),
 ('law', 0.013666032079834125),
 ('movement', 0.01263859173613297),
 ('human', 0.011336634746758036),
 ('power', 0.009767900564632248),
 ('class', 0.009448437032047294),
 ('india', 0.00883783344140136),
 ('war', 0.008653606664561548),
 ('justice', 0.008545826498751454),
 ('force', 0.008378188299976308),
 ('group', 0.007653617420772371),
 ('case', 0.007491521312487972),
 ('democracy', 0.007458441408911768),
 ('court', 0.006547614025984598),
 ('nation', 0.00653327255127553),
 ('rule', 0.006403078526256561)]

In [133]:
ldaseq.print_topics(time=2)[0]

[('state', 0.026444255569787172),
 ('right', 0.022891275618666593),
 ('social', 0.01886202830904104),
 ('political', 0.017980421567525422),
 ('society', 0.014172587790900745),
 ('law', 0.012539738361125971),
 ('movement', 0.012048150916955231),
 ('human', 0.010626244309854219),
 ('power', 0.009988567358716092),
 ('class', 0.009985669600555082),
 ('india', 0.00915290235683335),
 ('justice', 0.008979698744804166),
 ('war', 0.008481327715031812),
 ('force', 0.007862484339792834),
 ('democracy', 0.007811635745863406),
 ('group', 0.007634094079539142),
 ('nation', 0.007262372586181293),
 ('case', 0.007213855728332915),
 ('court', 0.0066895964177905495),
 ('rule', 0.006466330118256268)]

In [143]:
ldaseq.print_topics(time=9)[0]

[('state', 0.029367456257529775),
 ('right', 0.02713087846499159),
 ('law', 0.01749739156436228),
 ('political', 0.016990186271801745),
 ('social', 0.013424223868597632),
 ('human', 0.011547972683746648),
 ('power', 0.01135330351551832),
 ('india', 0.011141573889314577),
 ('court', 0.01065909961696536),
 ('society', 0.010424831658484658),
 ('public', 0.009192143285388648),
 ('justice', 0.008881724952934452),
 ('case', 0.00867759577680146),
 ('nigeria', 0.008510653610853031),
 ('nation', 0.008248423255607729),
 ('democracy', 0.008183484520902508),
 ('rule', 0.0081653398682865),
 ('act', 0.007370860741979389),
 ('national', 0.007021859681506771),
 ('group', 0.006487481101257665)]

In [134]:
docs_per_time_slice

[634, 1119, 1613, 2036, 2806, 3406, 8282, 9134, 8699, 7699]

In [135]:
ldaseq.save("ldaseqmodel")

In [136]:
loaded_model = ldaseqmodel.LdaSeqModel.load("ldaseqmodel")

In [144]:
list(np.arange(0.01, 1, 0.3))

[0.01, 0.31, 0.61, 0.9099999999999999]